###Installing and importing mysql.connector/python

In [55]:
!pip install --user mysql-connector-python

In [0]:
import mysql.connector

###Creating db connection and storing in capdb

In [0]:
 capdb = mysql.connector.connect(host='cpanel.insaid.co', user = 'student', password = 'student', database = 'Capstone1')

###Creating python cursor for executing mysql queries, checking tables, structures and count

In [0]:
capdbCursor = capdb.cursor()

In [59]:
capdbCursor.execute("show tables")
capdbTables = capdbCursor.fetchall()
for table in capdbTables:
    print(table)

('events_data',)
('gender_age_train',)
('phone_brand_device_model',)


In [60]:
capdbCursor.execute("describe events_data")
print(capdbCursor.fetchall())

[('event_id', 'bigint(20)', 'NO', '', None, ''), ('device_id', 'bigint(255)', 'YES', '', None, ''), ('timestamp', 'datetime', 'NO', '', None, ''), ('longitude', 'float(8,6)', 'YES', '', None, ''), ('latitude', 'float(8,6)', 'YES', '', None, ''), ('city', 'varchar(50)', 'NO', '', None, ''), ('state', 'varchar(50)', 'NO', '', None, '')]


In [61]:
capdbCursor.execute("select count(*) from events_data where state = 'delhi'")
capdbRecords = capdbCursor.fetchall()
for record in capdbRecords:
    print(record)

(751733,)


In [62]:
capdbCursor.execute("describe phone_brand_device_model")
print(capdbCursor.fetchall())

[('device_id', 'bigint(255)', 'NO', '', None, ''), ('phone_brand', 'varchar(2555)', 'NO', '', None, ''), ('device_model', 'varchar(2555)', 'NO', '', None, '')]


In [63]:
query =   """select count(*)
            from phone_brand_device_model m
            left join events_data e
            where e.state = 'delhi'"""
capdbCursor.execute(query)
capdbRecords = capdbCursor.fetchall()
for record in capdbRecords:
    print(record)

TypeError: ignored